<a href="https://colab.research.google.com/github/caxaxa/Chacha_PhD_Projects/blob/master/My_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

#Download kaggle.json
# more info on: https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

In [ ]:
# Transfer to Kaggle dircetorz
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
#Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Loading Dataset
! kaggle datasets download rvasel/qconcursos

In [ ]:
#Unziping
!unzip '/content/qconcursos.zip'

In [ ]:
#Enabling GPUs

#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
type_dict = {'created_at' : 'str',
'commented_by_professor'    :'int',
'difficulty'                :'int',
'discipline_id'             :'int',
'examining_board_id'        :'int',
'knowledge_area_id'         :'int',
'modality_id'               :'int',
'nullified'                 :'int',
'outdated'                  :'int',
'product_id'                :'int',
'institute_id'              :'int',
'publication_year'          :'int',
'scholarity_id'             :'int'}

In [ ]:
#Building the datasets
import pandas as pd

df_model = pd.read_csv('/content/Dataset_model.csv')
df_questions = pd.read_csv('/content/subjects_questions.csv')
sub_example  = pd.read_csv('/content/Submit.csv',sep =';')

In [ ]:
#map the subjecti id from novo_question_id
#Note, rearrage the columns before turning into dict
df_model['subject_id'] = df_model['novo_question_id'].map(df_questions[['novo_question_id','subject_id']].to_dict())

In [ ]:
#getting rid of NaNs

integers = ['novo_user_id','commented_by_professor','difficulty','discipline_id','examining_board_id', 'knowledge_area_id',
       'modality_id', 'nullified', 'outdated', 'product_id','institute_id',
       'publication_year', 'scholarity_id', 'novo_question_id','row']

df_model[integers]= df_model[integers].fillna(1000) 
df_model = df_model.fillna('NAO CONSTA')

In [ ]:
#setting correct dtypes

df_model = df_model.astype(type_dict)

In [ ]:
#feature created at to timestamp
#change datetime to timestamp
#df_model['created_at'] = df_model[['created_at']].apply(lambda x: x[0].datetime(), axis=1).astype()
df_model = df_model.drop(['created_at'],axis = 1)


# Building and encoding features and target on Keras

In [ ]:
#Importing Libraries
import gc
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Building and encoding features and target on Keras

val_dataframe = df_model.sample(frac=0.2, random_state=1337)
train_dataframe = df_model.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

In [ ]:

def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("acertou")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [ ]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

In [ ]:
#Batching Dataset
batch_size = 500
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

## Building one-hot encode functions

In [ ]:
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup


def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

In [ ]:
inputs = df_model.columns.tolist()
del df_model
gc.collect()
#with the following features being categorical:

categoricals  =  integers

as_string =  [x for x in inputs if x not in integers]
as_string.remove('acertou')
#exclude = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68','target']



#list of all numerical 

#apparently there is no simpler sinthax to remove list of elements from a list

#numericals = [x for x in inputs if x not in exclude]

# Building the model

In [ ]:
# Categorical features encoded as integers
var_inputs= {}
var_encoded = {}
# for var_num in numericals:
#     var_inputs[var_num] =  keras.Input(shape=(1,), name=var_num)
#     var_encoded[var_num] = encode_numerical_feature(var_inputs[var_num], var_num, train_ds)

In [ ]:
# Categorical feature encoded as string

for var_num in as_string:
    var_inputs[var_num] =  keras.Input(shape=(1,), name=var_num, dtype="string")
    var_encoded[var_num] = encode_categorical_feature(var_inputs[var_num], var_num, train_ds, True)

In [ ]:
# Categorical features encoded as integers

for var_num in categoricals:
    var_inputs[var_num] =  keras.Input(shape=(1,), name=var_num, dtype="int64")
    var_encoded[var_num] = encode_categorical_feature(var_inputs[var_num], var_num, train_ds, False)

In [ ]:
inputs.remove('acertou')

input_list = []
for i in inputs:
    input_list.append(var_encoded[i])

In [ ]:
all_features = layers.concatenate(input_list)

x = layers.Dense(batch_size, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)

In [ ]:
model = keras.Model(var_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"]) #try to build the f1 stat 02*(precision/recall/(preisionrecall)) see on : https://www.tensorflow.org/api_docs/python/tf/keras/metrics

In [ ]:
# `rankdir='LR'` is to make the graph horizontal.
#keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

In [ ]:
model.save('my_model.tf')

In [ ]:
model.summary()